<a href="https://colab.research.google.com/github/choiitea/Choi_DSPN_S24/blob/main/exercise12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 12: Cross validation
-----

In this exercise, we'll practice implementing cross validation techniques, including leave-one-out and k-fold cross validation. We'll use the `PimaIndiansDiabetes2` practice dataset, which has medical data on a group of Pima Native American women, including whether or not they have diabetes. This dataset is part of the `mlbench` package. We'll be using each person's medical history to predict whether or not they have been diagnosed with diabetes.

# 1: Data (1 pts)
---

Load the `tidyverse`, `boot`, and `mlbench` packages (you may need to install `boot` and `mlbench`).

Load the `PimaIndiansDiabetes2` dataset using the `data()` function. Drop the `insulin` column (it just has a lot of missing data) and then drop `NA`s from the rest of the dataset. Save your updated dataset to a new variable name. Finally, print the dimensions of your new dataset, and look at the first few lines of data.

In [21]:
# INSERT CODE HERE
library(tidyverse)
library(boot)
#install.packages("mlbench")
library(mlbench)

data("PimaIndiansDiabetes2")
data_clean <- drop_na(PimaIndiansDiabetes2)

dim(data_clean)
head(data_clean)

[1] 392   9

,pregnant,glucose,pressure,triceps,insulin,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
4,1,89,66,23,94,28.1,0.167,21,neg
5,0,137,40,35,168,43.1,2.288,33,pos
7,3,78,50,32,88,31.0,0.248,26,pos
9,2,197,70,45,543,30.5,0.158,53,pos
14,1,189,60,23,846,30.1,0.398,59,pos
15,5,166,72,19,175,25.8,0.587,51,pos


(Note that in medical contexts, `pedigree` refers to a system of measuring family history of a condition. So here, higher numbers mean greater family history of diabetes. You can read more about this dataset [here](https://rdrr.io/cran/mlbench/man/PimaIndiansDiabetes.html).)

# 2. Leave-one-out Cross Validation (4 pts)

In the tutorial, we learned how to fit leave-one-out cross validation using the `cv.glm` function from the `boot` package. But we can also do this manually using `predict()` like we have in the past.

Let's predict `diabetes`, a dichotomous outcome, using all the other variables in our modified dataset.

First, fit a logistic regression model using all of the observations except the very first one. Then use your fitted model to predict whether your holdout case is positive or negative for diabetes. Remember that logistic regression coefficients are in **log-odds**, meaning that if an output is positive, the probability of the outcome is greater than 50%; if the output is negative, the probability of the outcome is less than 50%.

Compare your result to the actual response in row one above. Did your model correctly classify this observation?

In [24]:
# INSERT CODE HERE
log_model <- glm(diabetes ~ ., data = data_clean[-1,], family = binomial)
# predict the probabilty of diabetes response given all variables
log_model_pred <- predict(log_model, newdata = data_clean[1,], type = "response")
# classify the prediction
log_model_pred_status <- ifelse(log_model_pred > 0.5, "positive", "negative")
actual_status <- ifelse(data_clean$diabetes[1] == "pos", "positive", "negative")
log_model_pred_status
actual_status
print("Yes, the model correctly classified this observation")

4 
"negative"

[1] "negative"

[1] "Yes, the model correctly classified this observation"


So we just calculated a single iteration of LOOCV. We used 531 rows of our data to fit a model to predict the outcome of the last row.

Below, use a `for` loop to iterate through the rest of your dataset doing the same thing. You will need to:
* Create a data frame `results` with two columns: one named `actual` which holds the true classification for each observation, and one named `predicted`, which should be filled with `NA`s. This is where you'll store the output of your loop.
* Create a loop that runs through each row of your data, pulls that observation out, trains your model on the remaining data, and then tests the fitted model on your test observation.
* Store your model *predictions* ("pos" or "neg" -- not the log-odds) in the `predicted` column of your `results` dataframe

After you run your loop, print the first few lines of `results`.

In [25]:
# Initialize `results` data frame
results <- data.frame(actual = data_clean$diabetes, predicted = rep(NA, nrow(data_clean)))

# INSERT CODE HERE

#for loop
for (i in 1:nrow(data_clean)){ #don't forget to change this to your data set name
    # separate individual observation `i` from the rest of your data
    # INSERT CODE HERE
    test_data <- data_clean[i, ]
    train_data <- data_clean[-i, ]

    # train your model
    # INSERT CODE HERE
    model <- glm(diabetes ~ ., data = train_data, family = "binomial")

    # test model on hold out observation
    # INSERT CODE HERE
    pred <- predict(model, newdata = test_data, type = "response")

    # classify model prediction as "pos" or "neg" and add to `results`
    # INSERT CODE HERE
    pred_status <- ifelse(pred > 0.5, "pos", "neg")
    results$predicted[i] <- pred_status

}

head(results)

,actual,predicted
,<fct>,<chr>
1,neg,neg
2,pos,pos
3,pos,neg
4,pos,pos
5,pos,pos
6,pos,pos


Now, calculate the overall error of your model. What proportion of cases were incorrectly classified?

In [26]:
# INSERT CODE HERE
error_rate <- sum(results$actual != results$predicted) / nrow(results)
print(error_rate)


[1] 0.2219388


# 3. Compare to `cv.glm` (3 pts)

Now, let's compare this result to the `cv.glm` function. Using the tutorial as a guide, use `cv.glm` to run LOOCV on the data, using the same model (i.e., still using all of the variables to predict diabetes diagnosis).

Note that, because this is a `classification` problem and not a regression problem like in the tutorial, we need to adjust the `cost` argument of `cv.glm`. We can read more about this in the docs:

In [27]:
?cv.glm

Here, we see `cost` is defined as:
> "A function of two vector arguments specifying the cost function for the cross-validation. The first argument to cost should correspond to the **observed responses** and the second argument should correspond to the **predicted or fitted responses** from the generalized linear model."

In the example code (scroll to bottom of the docs), we see that the appropriate cost function for a binary classification is

``
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
``

Where `r` is the vector of observed responses (technically "pos" and "neg", but R treats these as 1 and 0 under the hood), and `pi` is the vector of *probabilities* (not log-odds) fit by the model. Thus, this boils down to our error: what proportion of observations were incorrectly classified. You will need to include this code below.

In [28]:
# INSERT CODE HERE
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
model <- glm(diabetes ~ ., data = data_clean, family = "binomial")

cv_result <- cv.glm(data_clean, model, cost, K = nrow(data_clean))
cv_result$delta


[1] 0.2219388 0.2217566

How do your results compare to your manual LOOCV above?

> * *Write response here*
>
> * about the same results.


# 4. Adjusting K and Reflection (2 pts)

Recall that LOOCV has some drawbacks. In particular, it has quite high *variance* which can lead to poor performance on new test data. We can reduce this variance by increasing K.

Below, re-run your cross validation using `cv.glm` with `k` set to 3, 5, 10, and 15.

In [29]:
set.seed(1)
#INSERT CODE BELOW
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
model <- glm(diabetes ~ ., data = data_clean, family = "binomial")

# K = 3
cv_result_k3 <- cv.glm(data_clean, model, cost, K = 3)
cv_result_k3$delta

# K = 5
cv_result_k5 <- cv.glm(data_clean, model, cost, K = 5)
cv_result_k5$delta

# K = 10
cv_result_k10 <- cv.glm(data_clean, model, cost, K = 10)
cv_result_k10$delta

# K = 15
cv_result_k15 <- cv.glm(data_clean, model, cost, K = 15)
cv_result_k15$delta

[1] 0.2168367 0.2253618

[1] 0.2168367 0.2229605

[1] 0.2168367 0.2216394

[1] 0.2193878 0.2248217

#### Reflection

How do your errors compare to your LOOCV error above? How do they change as k increases?
> * *Write response here*
>
> *  Generally, the LOOCV error rate and k-fold CV error rates are highly similar. Looking at the raw cross-validation estimates (first column), the error rate is equal for k = 3, k = 5, and k = 10, and these k-fold CV error rates are lower in error rate by 0.51%. Additionally, the LOOCV error rate is closer to the 10- and 15-fold CV error rates. This, however, is expected because LOOCV is an extreme case of k-fold CV where k is the number of observations. The small difference between the k-fold CV error rates and LOOCV error rates suggest that k-fold CV with smaller k value is a reasonable cross-validation method.

If you change the random seed above, you'll get slightly different errors. If you were to do the same with your LOOCV above , would you expect to get different results each time? Why or why not?
> * *Write response here*
> * the random seed in k-fold CVs affect how the data is partitioned, and therefore, different random seeds can affect the results slightly. However, for LOOCV, changing the random seed won't affect the results at all because LOOCV doesn't have any random partitioning of the data - every single iteration in the cross-validation simply leaves out one obversation at a time for n times (n being nunber of observations).


**DUE:** 5pm March 25, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*
>
>
